In [ ]:
import numpy as np
from scipy.integrate import nquad
from scipy.constants import c, epsilon_0, mu_0, hbar, Boltzmann, e, physical_constants

from arc import Rubidium85
from pint import UnitRegistry, set_application_registry

si = UnitRegistry()
set_application_registry(si)

c = c * si.meter / si.second
mu_0 = mu_0 * si.newton / si.ampere**2
epsilon_0 = epsilon_0 * si.farad / si.meter
hbar = hbar * si.joule * si.second
bohr_radius = physical_constants['Bohr radius'][0] * si.meter 
e = e * si.coulomb
atom = Rubidium85()

We want to find Rydberg transitions that have splittings within some experimentally accessible frequency range. 

In [ ]:
def find_suitable_transitions(frequency_bounds, max_n=100):
    suitable_transitions = []
    # nS -> nP transitions
    for i in range(6, max_n):
        for j in range(i+1, max_n):
            l1 = 2
            l2 = 3
            j1 = 2.5
            j2 = 3.5

            frequency = np.abs(atom.getTransitionFrequency(j, l1, j1, i, l2, j2)) * si.hertz
            if frequency > frequency_bounds[0] and frequency < frequency_bounds[1]:
                suitable_transitions.append([j, i, frequency])

    return suitable_transitions

def get_transition_parameters(n1, n2, temperature_K=300):
    l1 = 2
    l2 = 3
    j1 = 2.5
    j2 = 3.5
    mj1 = j1
    mj2 = j2
    
    dipole_matrix_element = atom.getDipoleMatrixElement(n1, l1, j1, mj1, n2, l2, j2, mj2, s=0.5, q=1)
    dipole_matrix_element_SI = dipole_matrix_element * bohr_radius * e

    tau1 = atom.getStateLifetime(n1, l1, j1, temperature_K, includeLevelsUpTo=150) * si.second
    tau2 = atom.getStateLifetime(n2, l2, j2, temperature_K, includeLevelsUpTo=150) * si.second

    Gamma_p = 0.5 * (1.0/tau1  + 1.0/tau2) * 2 * np.pi
    return dipole_matrix_element_SI, Gamma_p

suitable_transitions = [{"n1": t[0], "n2": t[1], "frequency": t[2]} for t in find_suitable_transitions([10 * si.gigahertz, 15 * si.gigahertz])]
for transition in suitable_transitions:
    dipole_matrix_element, Gamma_p = get_transition_parameters(transition["n1"], transition["n2"])
    transition["dipole_matrix_element"] = dipole_matrix_element.to(si.debye)
    transition["Gamma_p"] = Gamma_p.to(si.hertz)
    print(f"Transition {transition["n1"]}D -> {transition["n2"]}F at {transition["frequency"]:#~P} with dipole matrix element {transition["dipole_matrix_element"]:#~P} and Gamma_p {transition["Gamma_p"]:#~P}")

Transition 54S -> 53P at 14.723740706248611 GHz with dipole matrix element -6.212256317590297 kD and Gamma_p 85.99521891024114 kHz
Transition 55S -> 54P at 13.918435177654391 GHz with dipole matrix element -6.448119554294987 kD and Gamma_p 82.02938821348332 kHz
Transition 56S -> 55P at 13.170797704226624 GHz with dipole matrix element -6.688371356017867 kD and Gamma_p 78.32091984458822 kHz
Transition 57S -> 56P at 12.47575773353525 GHz with dipole matrix element -6.9330117254273596 kD and Gamma_p 74.84820511131983 kHz
Transition 58S -> 57P at 11.828770402015671 GHz with dipole matrix element -7.182040665740189 kD and Gamma_p 71.59247768904488 kHz
Transition 59S -> 58P at 11.225754038777184 GHz with dipole matrix element -7.435458179068937 kD and Gamma_p 68.53631508013063 kHz
Transition 60S -> 59P at 10.663036017798007 GHz with dipole matrix element -7.693264268090247 kD and Gamma_p 65.66422400969638 kHz
Transition 61S -> 60P at 10.137305724662866 GHz with dipole matrix element -7.95545

Now, we can pick one of these transitions (and some TE mode (m,n,p)) and design a cavity for it. The cavity will have rectangular dimensions A x B x C, defined by the frequency / wavelength of the transition:
$$\lambda_c = \frac{c}{\nu_c}$$

$$\nu_c = \frac{c}{2} \sqrt{\left(\frac{m}{A}\right)^2 + \left(\frac{n}{B}\right)^2 + \left(\frac{p}{C}\right)^2}$$

There are infinitely many choices of dimensions for any given wavelength, but we can fix some aspect ratio to simplify the choices.


In [37]:
def cavity_dims_from_aspect_ratios(f_c, m, n, p, r_ab=1.0, r_db=1.3):
    K = 2.0*f_c/c
    num = (m/r_ab)**2 + n**2 + (p/r_db)**2
    b = np.sqrt(num)/K
    a = r_ab*b
    d = r_db*b
    return a.to(si.meter), b.to(si.meter), d.to(si.meter)

transition = max(suitable_transitions, key=lambda t: np.abs(t['dipole_matrix_element']))
f_c = transition['frequency']

print (f"Frequency: {f_c:#~P}, Dipole matrix element: {transition['dipole_matrix_element']:#~P}")

m, n, p = 1, 0, 1
a, b, d = cavity_dims_from_aspect_ratios(f_c, m, n, p)

print (f"a={a:#~P}\nb={b:#~P}\nd={d:#~P}")

kx = m * np.pi / a
ky = n * np.pi / b
kz = p * np.pi / d


Frequency: 10.137305724662866 GHz, Dipole matrix element: -7.9554589348895774 kD
a=18.655244726432134 mm
b=18.655244726432134 mm
d=24.251818144361778 mm


According to "[Generalized effective mode volume for leaky optical cavities](https://opg.optica.org/ol/fulltext.cfm?uri=ol-37-10-1649&id=233159)", effective mode volume is:

$$V_{\mathrm{eff}}^{\mathrm{N}}(\mathbf{r}_{\mathrm{c}})=\frac{\int_V \epsilon_{\mathrm{r}}(\mathbf{r})\left|\tilde{\mathbf{f}}_{\mathrm{c}}(\mathbf{r})\right|^2  \mathrm{~d} \mathbf{r}}{\epsilon_{\mathrm{r}}\left(\mathbf{r}_{\mathrm{c}}\right)\left|\tilde{\mathbf{f}}_{\mathrm{c}}\left(\mathbf{r}_{\mathrm{c}}\right)\right|^2}$$

In vacuum, our permittivity is constant, so we have:

$$= V_{\mathrm{eff}}^{\mathrm{N}}(\mathbf{r}_{\mathrm{c}})=\frac{\int_V \left|\tilde{\mathbf{f}}_{\mathrm{c}}(\mathbf{r})\right|^2  \mathrm{~d} \mathbf{r}}{\left|\tilde{\mathbf{f}}_{\mathrm{c}}\left(\mathbf{r}_{\mathrm{c}}\right)\right|^2}$$

The cavity mode has some prefactors but we can ignore them because they will cancel in the effective mode volume calculation.

In [ ]:
# annoyingly the numerical integration cannot handle units
# make sure everything is in meters
def TE_cavity_mode_normalized(kx, ky, kz):
    Ex = lambda x, y, z: (np.cos(kx.to(1 / si.meter).magnitude * x) * np.sin(ky.to(1 / si.meter).magnitude * y) * np.cos(kz.to(1 / si.meter).magnitude * z))
    Ey = lambda x, y, z: (np.sin(kx.to(1 / si.meter).magnitude * x) * np.cos(ky.to(1 / si.meter).magnitude * y) * np.sin(kz.to(1 / si.meter).magnitude * z))
    Ez = lambda x, y, z: 0
    return [Ex, Ey, Ez]

def TE_cavity_mode_intensity(kx, ky, kz):
    Ex, Ey, Ez = TE_cavity_mode_normalized(kx, ky, kz)
    I = lambda x, y, z: Ex(x, y, z)**2 + Ey(x, y, z)**2 + Ez(x, y, z)**2
    return I

def box_integral_nquad(kx, ky, kz, a, b, d, epsabs=1e-10, epsrel=1e-10):
    f = TE_cavity_mode_intensity(kx, ky, kz)
    res, err = nquad(f, [[0, a.to(si.meter).magnitude], [0, b.to(si.meter).magnitude], [0, d.to(si.meter).magnitude]], opts={'epsabs': epsabs, 'epsrel': epsrel})
    return res, err

# this is a function of (x,y,z)
def V_eff(kx, ky, kz, a, b, d):
    IE, IE_err = box_integral_nquad(kx, ky, kz, a, b, d)
    I_c = TE_cavity_mode_intensity(kx, ky, kz)
    return lambda x, y, z: (IE * si.meter**3) / I_c(x.to(si.meter).magnitude, y.to(si.meter).magnitude, z.to(si.meter).magnitude)

cavity_center = (a/2, b/2, d/2)
cavity_Veff = V_eff(kx, ky, kz, a, b, d)
Veff = cavity_Veff(cavity_center[0], cavity_center[1], cavity_center[2])
print ("Effective mode volume at cavity center: ", Veff.to(si.millimeter**3))

Effective mode volume at cavity center:  2110.0182563680787 millimeter ** 3


The zero-point electric field $E_{zpf}$ is given by:
$$E_{zpf}(\mathbf{r}_{\mathrm{c}}) = \sqrt{\frac{\hbar \omega_c}{2 \epsilon_0 V_{eff}}}$$

where $\omega_c$ is the cavity resonance frequency, $\epsilon_0$ is the vacuum permittivity, and $V_{eff}$ is the effective mode volume. 

The single-atom coupling rate $g_0$ is given by:
$$g_0(\mathbf{r}_{\mathrm{c}}) = \frac{\mu E_{zpf}(\mathbf{r}_{\mathrm{c}})}{\hbar \eta_{pol}}$$

where $\mu$ is the electric dipole matrix element for the given Rydberg transition (nS -> mP etc), from ARC, and $\eta_{pol}$ is the polarization overlap (which we may assume to be 1). 

In [106]:
def E_zpf(omega_c, cavity_Veff):
    return lambda x,y,z: np.sqrt(hbar * omega_c / (2 * epsilon_0 * cavity_Veff(x,y,z)))

# needs units
def g_0(mu, omega_c, cavity_Veff, eta_pol=1):
    return lambda x,y,z: mu * E_zpf(omega_c, cavity_Veff)(x,y,z) / (hbar * eta_pol)

mu = np.abs(transition['dipole_matrix_element'])
omega_c = 2 * np.pi * f_c
print (E_zpf(omega_c, cavity_Veff)(cavity_center[0],cavity_center[1],cavity_center[2]).to(si.millivolt / si.meter))
print (g_0(mu, omega_c, cavity_Veff)(cavity_center[0],cavity_center[1],cavity_center[2]).to(si.kilohertz))


0.4239911752442864 millivolt / meter
106.69036276042222 kilohertz


The last thing we would like to calculate is required driving power for the transition from the 5P state to the Rydberg state. This can be done through ARC.

In [123]:
driving_power = atom.getDrivingPower(
    n1 = 5, # --> 5P
    l1 = 1, # 5P <--
    j1 = 1.5, # 1 unit of orbital and 0.5 of spin
    mj1 = 1.5, # degenerate? 
    n2 = transition['n1'], # the lower Rydberg state
    l2 = 2, # we just assumed this
    j2 = 2.5,
    mj2 = 2.5,
    q = 1, # right-hand circular polarization, also arbitrary
    rabiFrequency = 5e6, # this is the Rabi frequency we want
    laserWaist = 0.002, # 2mm
)

print (driving_power * si.watt) 

0.23942603675121082 watt
